# Harmonic Signals
### Pitch and Timbre of Periodic Signals (sine, square, sawtooth, ...)

Author: Dirk Van Compernolle   
Created: 29/03/2021

In [1]:
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, interactive_output
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from IPython.display import display, clear_output, Audio, HTML
import time
import librosa
import pyspch.spectrogram as specg
import pyspch.display as spch_disp

def my_sine(x, w, amp, phi):
    return amp*np.sin(w * (x-phi))
#
def synth(sigtype='sin', freq=200.0, amp=1.0, phi=0.0, sample_rate=8000, Tmax=0.25):
    t = np.linspace(0.0, Tmax, int(Tmax*sample_rate), endpoint=False)
    tt = 2.*np.pi*(freq*t) + phi
    if sigtype == 'sin':
        y = np.sin(tt)
    elif sigtype == 'square':
        y = signal.square(tt)
    elif sigtype == 'sawtooth':
        y = signal.sawtooth(tt)
    elif sigtype == 'chirp 1:20':
        y = signal.chirp(t,freq,Tmax,20.*freq,method='linear')
    elif sigtype == 'chirp 20:1':
        y = signal.chirp(t,20*freq,Tmax,freq,method='linear')
    elif sigtype == 'gausspulse':
        y = signal.gausspulse(t-Tmax/2.,fc=freq,bw=.4)
    elif sigtype == 'modulated white noise':
        y = np.random.randn(len(t))*np.sin(tt)
    else:
        print( 'signal: Unrecognized signal type')
    return amp*y, t


# a default boxed layout
def box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

# The following code will increase the default width of your Jupyter notebook cells
# Supposed to work well 
display(HTML(data="""
<style>
    div#notebook-container    { width: 99%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [2]:
class Harmonic_Signals(widgets.HBox):

    def __init__(self,Tmax=.3,sample_rate=16000,freq_range=[100.,500.]):
        super().__init__()
        
        if sample_rate < 4000.:
            print("WARNING: sampling rate should not be lower than 4000.0 Hz and has been reset.")
            sample_rate = 4000.0
        self.sample_rate = sample_rate
        self.nfft = 512
        self.Tmax = Tmax        
        display_types = ['spectrum','spectrogram']
        signal_types = [ 'sin', 'square', 'sawtooth', 'chirp 1:20','chirp 20:1', 'gausspulse','modulated white noise']
        
        self.disptype = 'spectrum'
        self.sigtype = 'sin'
        self.freq = 110.
        self.amp = 1.
        self.phi = 0.
        self.autoplay = False

        # create the widgets
        self.wg_disptype = widgets.Dropdown(options=display_types,value=self.disptype,description="Display")
        self.wg_sigtype = widgets.Dropdown(options=signal_types,value=self.sigtype,description="Signal")
        self.wg_amp = widgets.FloatLogSlider(value=self.amp,step=0.2,min=-2.,max=0.0,description='Amplitude',continous_update=False)
        freq_step = 10. if (freq_range[1]-freq_range[0]) > 50 else 1.  
        self.wg_freq = widgets.FloatSlider(value=self.freq,step=freq_step,min=freq_range[0],max=freq_range[1],description='Frequency',continous_update=False)
        self.wg_phi = widgets.FloatSlider(value=self.phi,step=0.1,min=0.0,max=2.*np.pi,description='Phase',continous_update=False)
        self.wg_autoplay = widgets.Checkbox(value=self.autoplay,description='Autoplay Audio',indent=False,button_style='warning')
        self.wg_autoplay.layout.width='50%'
        self.wg_clear_log = widgets.Button(description='Clear the log')
        self.wg_clear_log.layout.width='50%'
        
        # link to the widget observers
        self.wg_disptype.observe(self.disptype_observe,'value')
        self.wg_sigtype.observe(self.sigtype_observe,'value')
        self.wg_amp.observe(self.amp_observe,'value')    
        self.wg_freq.observe(self.freq_observe,'value')
        self.wg_phi.observe(self.phi_observe,'value')
        self.wg_autoplay.observe(self.autoplay_clicked, 'value')
        self.wg_clear_log.on_click(self.clear_log_clicked)

        # setup the outputs 
        self.audio = widgets.Output()
        self.logscr = widgets.Output()
        self.out = widgets.Output(layout=box_layout())
        self.UI = widgets.VBox( [self.wg_disptype,self.wg_sigtype, self.wg_amp, self.wg_freq, self.wg_phi,self.wg_autoplay],
                                 layout=box_layout())
        self.right_screen = widgets.VBox([self.UI, self.audio, self.wg_clear_log,  self.logscr],layout=box_layout())
                                          
        # initialize the plots
        #self.fig,self.ax = plt.subplots(2,1,constrained_layout=True, figsize=(9, 4))
        self.fig = spch_disp.make_subplots(row_heights=[1.,1.], figsize=(6, 4),dpi=100)        
        self.update()
        plt.close()          # avoids output of dummy figure on startup
        # initialize the displayed function

        # attach children to the VBox class
        self.children = [ self.out, self.right_screen ] 

    def update(self):
        y,x = synth(sigtype=self.sigtype,freq=self.freq,phi=self.phi,amp=self.amp,sample_rate=self.sample_rate,Tmax=self.Tmax)

        ax = self.fig.axes
       
        if self.disptype == 'spectrum':
            ax[0].cla()
            ax[1].cla()
            spch_disp.add_line_plot(self.fig,y,row=1,x=x,yrange=[-1.,1.],xlabel='Time(sec)',title='Waveform')
            freq_ax,spec = signal.periodogram(y,fs=self.sample_rate,scaling='spectrum')
            ax[1].plot(freq_ax,np.sqrt(2*spec))
            ax[1].set_title('Spectrum')
            ax[1].set_xlabel('Frequency (Hz)')
            ax[1].set_ylim([0, 1])
            ax[1].grid(True)
            ax[1].set_xlim([0,self.sample_rate/2.])
        elif self.disptype == 'spectrogram':
            spg = specg.spectrogram(y,sample_rate=self.sample_rate,preemp=0.)
            yax = np.arange(self.nfft/2 + 1)*(self.sample_rate/self.nfft) 
            spch_disp.plot_spg(spg,fig=self.fig,wav=y,sample_rate=self.sample_rate,yax=yax)
            ax[0].set_title('Waveform')
            ax[0].set_xlabel('')
            ax[1].set_title('Spectrogram')
            ax[1].set_xlabel('Time(sec)')
            #ax[1].set_ylabel('Hz')

            
        # here come the things that go to dedicated output widgets
        with self.out:
            clear_output(wait=True)
            display(self.fig)
        with self.audio:
            clear_output(wait=True)
            try:
                display(Audio(data=y,rate=self.sample_rate,normalize=False, autoplay=self.autoplay))
            except: 
                try:
                    with self.logscr:
                        #clear_output(wait=True)
                        print("Warning: playing normalized sound")
                    display(Audio(data=y,rate=self.sample_rate, autoplay=self.autoplay))
                except: pass
     
    def disptype_observe(self,change):
        self.disptype = change.new
        self.update()
    
    def sigtype_observe(self,change):
        self.sigtype = change.new
        self.update()
    
    def amp_observe(self,change):
        self.amp = change.new
        self.update()
        
    def freq_observe(self,change):
        self.freq = change.new
        self.update()
        
    def phi_observe(self,change):
        self.phi = change.new
        self.update()
        
    def autoplay_clicked(self,obj):
        self.autoplay = not self.autoplay
        with self.logscr: print('Toggling autoplay',self.autoplay) 

    def clear_log_clicked(self,b):
        with self.logscr: clear_output()                   



## Pitch and Timbre

### Pitch of Pure Tones

Pitch is often defined as the characteristic of a sound that makes it sound high or
low, or that determines its position on a musical scale. Pitch is related to the repetition
rate of the waveform of a sound. For a pure tone, this corresponds to the frequency; for
a 'complex tone it usually (but not always) corresponds to the fundamental frequency.
Frequency is the most important contributor to the sensation of pitch, but not the only
one by any means. Other contributors to pitch include intensity, spectrum, duration,
amplitude envelope, and the presence of other sounds.
Various attempts have been made to establish a psychophysical pitch scale. If, after
listening to a 4000-Hz tone followed by a tone of very low frequency, one is asked to
tune an oscillator to a pitch halfway between, a likely choice would be around 1000 Hz.
On a scale of pitch, then, 1000 Hz is judged halfway between 0 and 4000 Hz. The unit
for subjective pitch is the me/; the scale is arranged so that doubling the number of mels
doubles the subjective pitch. A scale from 0 to 2400 mels covers the audible range of
20 to 16,000 Hz.
A numerical scale of pitch (in mels) is not nearly so useful as a numerical scale
of loudness (in sones), however. Pitch is more often related to a musical scale, where
the octave is the "natural" pitch interval that is subdivided into the desired number of
steps.
Two major theories of pitch perception have been developed; they are usually referred to as the place (or frequency) theory and the periodicity (or time) theory. According to the place theory, the cochlea converts a vibration in time to a vibration pattern
in space (along the basilar membrane), and this in turn excites a spatial pattern of
neural activity. The place theory explains some aspects of auditory perception but fails
to explain others.
According to the periodicity theory of pitch, the ear performs a temporal analysis
of the sound wave. Presumably, the time distribution of impulses carried along the
35 
auditory nerve has encoded into it the temporal structure of the sound wave.

### Pitch of Complex Tones

One of the most remarkable properties of the auditory system is its ability to extract
pitch from complex tones. When the complex tone consists of a number of harmonically
related partials, the pitch corresponds to the "missing fundamental." This pitch is often
referred to as pitch of the missing fundamental, virtual pitch, or musical pitch.
When the partials are not exactly harmonics of a missing fundamental, we arrive at
a "virtual pitch" by some strategy that may weigh several possibilities, and when the
choice is difficult the pitch may be ambiguous.
Familiar examples of such virtual pitch are the bass notes we hear from loudspeakers
of very small size that radiate negligible power at low frequencies, and the subjective
strike note of carillon bells, tuned church bells and orchestral chimes.

### Timbre

The American National Standards Institute defines timbre as " ... that attribute of
auditory sensation in terms of which a listener can judge that two sounds similarly
presented and having the same loudness and pitch are dissimilar". According to this
definition, timbre is the subjective correlate of all those sound properties that do not
directly influence pitch or loudness. These properties include the sound's spectral power
distribution, its temporal envelope (as would be shown on an oscilloscope display), rate
and depth of amplitude or frequency modulation, and the degree of inharmonicity of
its partials. The timbre of a sound therefore depends on many physical variables. It
has been shown that from a subjective point of view the sensation of timbre has about
three rather orthogonal dimensions. These can be represented by the verbal ranges dullsharp, compact-scattered and colorful-colorless. These subjective dimensions are loosely
related to the physical quantities of high-frequency energy in the attack, synchronicity
in high-harmonic transients, and spectral power distribution.
The concept of timbre plays a very important role in the orchestration of traditional
music and in the composition of computer music. There is, however, no satisfying
comprehensive theory of timbre perception. Neither is there a uniform nomenclature
to designate or classify timbres. This poses considerable problems in communicating or
teaching the skills of orchestration and computer score writing to student-composers.
In the following demonstrations one can hear how spectral make-up, temporal envelope and degree of spectral inharmonicity all have a very specific influence on the
perceived timbre of sounds from musical instruments. 

(For a demo with musical instruments, see **Demo 28 in AuditoryDemonstrations**)

## Waveform and Spectrum of Harmonic Signals
##### The Analytical Perspective

In this demo periodic signals are generated that can be characterized as a harmonic complex of sine waves.

$y= A \sum_k \alpha_k  sin(2  \pi k f_0 t+\Phi)$

In addition signals can be generated with either time-varying amplitude (e.g. Gaussian pulse) or time-varying frequency (e.g. chirps). 

You can control:
- The fundamental frequency $f_0$  (named "frequency" in the control panel)  
- The overall amplitude $A$
- The relative amplitude of the harmonics $\alpha_k$ by choosing from a set of predefined harmonic functions
- The global phase $\Phi$  (you cannot control relative phase differences per harmonic component in this demo)

Signal Types to choose from:
- stationary and periodic
    + sine wave
    + square 
    + sawtooth
- non stationary signals:
    + chirp 1:20 : ramping up the frequency by a factor of 20
    + chirp 20:1 : ramping down the frequency by a factor of 20
    + gausspulse

    
Displays to choose from:
- spectrum: most appropiate for stationary signals
- spectrogram: essential for non-stationary signals

Things to investigate:
- First play around with (fundamental) frequencies in the speech range, i.e. Do the non-stationary signals produce a tonal effect ?
- Compare the perception of a chirp (ramping up from 500 Hz to 10kHz) for different sampling frequencies (8000, 16000, 48000). What is the most striking phenomenon and do you know it's cause?
- 
    
CAVEAT:   
+ The Autoplay function does not always work properly on Ipython < 6.0 [Google Colab] which has autonormalization by default in the display.Audio() function

In [3]:
Harmonic_Signals(Tmax=.3,sample_rate=8000,freq_range=[2.,500.]) 

Harmonic_Signals(children=(Output(layout=Layout(border='solid 1px black', margin='0px 10px 10px 0px', padding=…